# Задача №1

Классифицируйте ирисы при помощи метода K-NN.
Используйте материал из лекции (5.3.5. Алгоритм k-nearest neighbors) 

## 1.1 Загрузите [Iris Data Set](https://archive.ics.uci.edu/ml/datasets/iris)

Удобный способ сделать это использовать модуль [sklearn.datasets](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html#sklearn.datasets.load_iris)

Пример загрузки датесета sklearn можно найти в лекции в разделе:
Табличные данные -> Загрузка данных



In [16]:
import sklearn 
from sklearn.datasets import load_iris

# Download dataset 
data = load_iris(return_X_y = True) # Так же можно получить данные в Bunch(словарь) или pandas DataFrame

features = data[0] # Массив 150x4 
class_labels = data[1] # Массив из 150 элементов каждый элемент это число обозначающее класс к которому относиться данный ирис: 0, 1, 2
print("Данные",features.shape)
print("Номер класса",class_labels.shape)

Данные (150, 4)
Номер класса (150,)


Визуализируйте данные в виде таблицы. (см. п. лекции Табличные данные ->Визуализация данных)


In [17]:
import pandas as pd 

data_bunch = load_iris(return_X_y = False)
#print(data_bunch.keys())

df = pd.DataFrame(data_bunch.data, columns=data_bunch.feature_names)
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


Визуализируйте данные из iris dataset при помощи Tensorboard. 

In [18]:
# Вспомогательный метод для запуска Tensorboard в Colab

# Fix https://stackoverflow.com/questions/60730544/tensorboard-colab-tensorflow-api-v1-io-gfile-has-no-attribute-get-filesystem
import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

import os
# Запуск Tensorboard в Colab
def reinit_tensorboard(clear_log = True):
  # Лог-файлы читаются из этого каталога: 
  logs_base_dir = "runs"
  if clear_log:
    # Очистка логов
    !rm -rfv {logs_base_dir}/*
    os.makedirs(logs_base_dir, exist_ok=True)
  # Магия Colab
  %load_ext tensorboard
  %tensorboard --logdir {logs_base_dir}

Используйте вкладку projector для визуализации данных 

In [19]:
from torch.utils.tensorboard import SummaryWriter
import numpy

reinit_tensorboard()

writer = SummaryWriter(comment = "iris")
np_f = numpy.array(features)
writer.add_embedding(np_f, metadata=class_labels )
writer.close()

removed 'runs/Jan26_15-14-42_55736e634746iris/projector_config.pbtxt'
removed 'runs/Jan26_15-14-42_55736e634746iris/events.out.tfevents.1611674082.55736e634746.723.0'
removed 'runs/Jan26_15-14-42_55736e634746iris/00000/default/tensors.tsv'
removed 'runs/Jan26_15-14-42_55736e634746iris/00000/default/metadata.tsv'
removed directory 'runs/Jan26_15-14-42_55736e634746iris/00000/default'
removed directory 'runs/Jan26_15-14-42_55736e634746iris/00000'
removed directory 'runs/Jan26_15-14-42_55736e634746iris'
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 85), started 3:17:54 ago. (Use '!kill 85' to kill it.)

<IPython.core.display.Javascript object>

## 1.2 Классифицируйте ирисы при помощи алгоритма Nearest Neighbor.

Алгоритм описан в лекции (Демонстрация работы с данными -> Изображения -> Алгоритм k-nearest neighbors)

Хотя в лекции дан пример для изображений, реализация для табличных данных будет проще, либо вообще может не отличаться от предложенной.

Желательно использовать методы из библиотеки pytorch, но допустимо выполнить задание используя numpy.


P.S. Nearest Neighbor это  K- Nearest Neighbor при K =1




In [20]:
from torch.utils.data import DataLoader
import torch
import random 
import numpy as np

class NearestNeighbor:
  def __init__(self):
    self.train_data = None
    self.train_labels = None

  def fit(self,x,y):
    """
      Обучение сводится к запоминанию примеров из тренировочного датасета в свойствах класса.
      Для выполнения задания модификация этого метода не требуется.
    """
    self.train_data = torch.vstack((self.train_data,x)) if self.train_data != None else x
    self.train_labels = torch.hstack((self.train_labels,y)) if self.train_labels != None else y
    
  def forward(self,x):
    """
    
         Arguments:
             x  (Tensor or numpy.array): набор объект из тестовой выборки (batch)
         
         Returns:
             class_num (Tensor or numpy.array) - вектор чисел соответствующих номерам 
             классов к которму относятся объекты их x.
             Реализация по умолчанию возвращает случайные числа.
    """
    class_count = torch.unique(self.train_labels).shape[0]
    class_num = torch.randint(low = 0, high = class_count, size = (x.shape[0],)) 
    return class_num
    
  
  def calculate_distances(self,x):
    return self.l1_metric(x)

  def l1_metric(self, x):
      """
        Arguments:
             x  (Tensor or numpy.array): набор объект из тестовой выборки (batch)
        Returns:
            distances - (Tensor or numpy.array): вектор растояний от каждго объекта из х до 
            всех объектов обучающей выборки
      """
      distances = []
      for k in range(len(x)):
        distances.append(np.abs((np.array(self.train_data) - x[k]).sum(axis=1)))
      # Put your code here

      # One loop allowed
      
      return distances

### 1.2.1 Убедитесь что реализация по умолчанию (класс NearestNeighbor) запускается с данными из iris dataset.

Отбратите внимание на тип входных данных. Пример приведения к типу torch.tensor можно найти в лекции в разделе "Демонстрация работы с данными->Последовательности->Обзор контейнеров".

Какую точность(accuracy) дает реализация по умолчанию?

In [21]:
model = NearestNeighbor()
model.fit(np_f, np.array(class_labels))
#print(model.calculate_distances(np_f))


### 1.2.2 Допишите недостающие части кода:


* В методе 
  
      predict
    
Вместо вектора случайных числ, возвращайте номера классов соответствующие объекту из обучающей выборки, для которого расстоянию  x будет минимальным.

* Рассчет расстояний по метрике L1 реализуйте в методе

      l1_metric

* Для выполнения задания рекомендуется использовать методы:

[torch.abs](https://pytorch.org/docs/stable/generated/torch.abs.html?highlight=abs#torch-abs)

[torch.sum](https://pytorch.org/docs/stable/generated/torch.sum.html?highlight=sum#torch-sum)

[torch.argmin](https://pytorch.org/docs/stable/generated/torch.argmin.html?highlight=argmin#torch-argmin)

либо их аналоги из библиотеки numpy


In [36]:
from torch.utils.data import DataLoader
import torch
import random 
import numpy as np

class NearestNeighbor:
    def __init__(self):
        self.train_data = None
        self.train_labels = None

    def fit(self, x, y):
        self.train_data = torch.vstack((self.train_data, x)) if self.train_data != None else x
        self.train_labels = torch.hstack((self.train_labels, y)) if self.train_labels != None else y

    def forward(self, x):
        class_num = []
        distances = self.calculate_distances(x)
        for k in range(len(x)):
            class_num.append(self.train_labels[np.array(distances).argmin(axis=1)[k]])
        return class_num

    def calculate_distances(self, x):
        return self.l1_metric(x)

    def l1_metric(self, x):
        distances = []
        for k in range(len(x)):
            distances.append((np.abs(np.array(self.train_data) - np.array(x)[k])).sum(axis=1))
        return distances

### 1.2.3 Подготовьте данные

Для этого разделите датасет на две части: тренировочную b проверочную.
Используйте для этого метод описанный в разделе лекции "Демонстрация работы с данными->Табличные данные->Обучение".

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, class_labels, test_size=0.2) # 80% training and 20% test

### 1.2.4 "Обучите" модель

Для этого просто вызовите метод fit передав ему на вход массив тренировочных данных с соответствующими метками классов. Аналогично тому как это демонстрировалось для в лекции ("Демонстрация работы с данными->Табличные данные->Обучение")

In [24]:
import numpy as np

model = NearestNeighbor()

model.fit(np.array(X_train), np.array(y_train))

### 1.2.5 Оцените точность модели по метрике accuracy

Демонстрация работы с данными->Табличные данные->Оценка точности

In [25]:
from sklearn import metrics
import numpy as np
y_pred = model.forward(np.array(X_test))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8666666666666667


# 2. Задача №2 Оцените точность работы реализованного в задании № 1 алгоритма. На датасете CIFAR10

## 2.1 Загрузите датасет [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html)


Рекомендуется использовать библиотеку [torchvision.datasets](https://pytorch.org/docs/stable/torchvision/datasets.html#cifar)

Пример загрузки вы можете найти в лекции в разделе (Демонстрация работы с данными->Работа с изображениями)





In [26]:
from torchvision import models, datasets, transforms

trainset = datasets.CIFAR10("content", train=True,  download=True)
valset = datasets.CIFAR10("content", train = False, download=True)

Files already downloaded and verified
Files already downloaded and verified


## 2.2 Создайте Dataloader и настройте предобработку данных

- Создайте объекты Dataloader и предобработаете данные при помощи механизма трансформаций.
Пример такой загрузки можно нейти в лекции в разделе " (Демонстрация работы с данными->Работа с изображениями)"

- Для ускорения работы допускается уменьшить размеры датасетов. Рекомендуемые  размеры 5000 изображений для тренировочного и 500 для проверочного. 
Для этой задачи можно использовать функцию [torch.utils.data.random_split](https://pytorch.org/docs/stable/data.html#torch.utils.data.random_split)
Пример ее использования есть в лекции в разделе: Демонстрация работы->Последовательности->Предобработка данных




In [37]:
from torch.utils.data import DataLoader
trainset.transform = transforms.Compose([transforms.ToTensor()])
train_dataloader = DataLoader(trainset, batch_size=5000, shuffle=True)
valset.transform = transforms.Compose([ transforms.ToTensor() ]) 
val_dataloader = DataLoader(valset, batch_size=500, shuffle=False)

## 2.3 Обучите классификатор(NearestNeighbor) на изображениях из CIFAR10


Возможно для этого вам придется внести изменения в реализацию которую вы сделали для Iris Data Set. В этом случае используйте наследование. Но вполне возможно этого избежать. Тогда код из ячейки ниже вам не понадобится. И вы сможете можете использовать класс NearestNeighbor из пункта 1 без модификаций. 

Проанализируйте в каком формате на вход модели подаются табличные данные и в каком виде Dataloader возвращает изображения.
Возможно вам удастся использовать метод [torch.flatten](https://pytorch.org/docs/stable/generated/torch.flatten.html?highlight=flatten#torch-flatten)




In [32]:
"""
 Используйте этот шаблон только есть он вам действительно нужен.
"""

class NearestNeighborForCIFAR10(NearestNeighbor):
  def __init__(self):
    super().__init__()
  

In [38]:
model_cifar = NearestNeighbor()
for img_batch, labels_batch in train_dataloader:
  model_cifar.fit(torch.flatten(img_batch, start_dim=1), labels_batch)
img_batch, class_num_batch = next(iter(val_dataloader))
#print(len(img_batch))
#print(model_cifar.train_data)
predicted_cls_nums = model_cifar.forward(torch.flatten(img_batch, start_dim=1))

##2.4 Оцените точность классификации изображений из CIFAR10 по метрике accuracy

Для итерации по проверочному датасету используйте Dataloader. Для расчета accuracy можно использовать sklearn аналогично тому как это сделано в задании №1. Или обратиться к примеру из лекции (Демонстрация работы с данными -> Изображения->Пример организации процесса обучения)

In [39]:
from sklearn import metrics
import numpy as np
print("Accuracy:",metrics.accuracy_score(class_num_batch, predicted_cls_nums))

Accuracy: 0.364


# 3. Задание №3

## 3.1 Реализуйте метод K-Nearest Neighbor.

Используйте материал из лекции (5.3.5. Алгоритм k-nearest neighbors).
Испоьзуйте наследование.
Метод рассчета расстояний остается без изменений, вам требуется переопределить только метод forward.
 

In [41]:
import numpy as np

class KNearestNeighbor(NearestNeighbor):
  def __init__(self,k):
    super().__init__()
    self.k = k
  
  def forward(self,x):
    class_num = []
    distances = np.array(self.calculate_distances(x))
    for n in range(len(distances)):
      class_num.append([])
    for _ in range(self.k):
      min_array = distances.argmin(axis=1)
      for n in range(len(distances)):
        class_num[n].append(self.train_labels[min_array[n]])
      for n in range(len(distances)):
        np.delete(distances[n], min_array[n])  
    predictions = []
    for n in range(len(distances)):
      predictions.append(max(set(class_num[n]), key=class_num[n].count))
    return predictions 

## 3.2 Протестируйте реализованный алгоритм со всеми значениями k из интервала [1 до 30]

Используете объекты Dataloader созданные в ходе выполнения задания №2
Постройте в Tensorboard график зависимости метрики accuracy от гиперпараметра k. Для записи в лог Tensorboard метод [add_scalar](https://pytorch.org/docs/stable/tensorboard.html?highlight=add_scalar#torch.utils.tensorboard.writer.SummaryWriter.add_scalar) 
Пример его использования которого можно найти в лекции (Демонстрация работы с данными->Последовательности->Попытка обучения)

При каких k достигается наилучшая точность?

In [43]:
from sklearn import metrics
import numpy as np
model_cifar_k = KNearestNeighbor(30)
for img_batch, labels_batch in train_dataloader:
  model_cifar_k.fit(torch.flatten(img_batch, start_dim=1), labels_batch)
img_batch, class_num_batch = next(iter(val_dataloader))
#print(len(img_batch))
#print(model_cifar.train_data)
predicted_cls_nums = model_cifar_k.forward(torch.flatten(img_batch, start_dim=1))
print("Accuracy:",metrics.accuracy_score(class_num_batch, predicted_cls_nums))

Accuracy: 0.364


# 4. Задание №4

## 4.1 Замените метрику расстояния с L1 на L2. 

При какой метрике расстояния и с каким k точность будет максимально точность будет максимальной?

Постройте графики точности для L1 и L2 в Tensorboard

In [ ]:
class KNearestNeighborL2(KNearestNeighbor):
  def __init__(self,k):
    super().__init__(k)
    self.k = k

  # Override
  def calculate_distances(self,x):
    # Заменим метрику L1 на L2
    return self.l2_metric(x)
  
  # Евклидово расстояние
  def l2_metric(self,x):
    """
        Arguments:
             x  (Tensor or numpy.array): набор объект из тестовой выборки (batch)
        Returns:
            distances - (Tensor or numpy.array): вектор растояний от каждго объекта из х до 
            всех объектов обучающей выборки
      """
      
      # Put your code here
      # One loop allowed
      
    return distances 

## 4.2 Постройте графики точности для L1 и L2 в Tensorboard

Обучите модель и посчитайте accuracy аналогично тому как это делалось в задании № 3. Для отображения двух графиков вместе не очищайте логи Tensorboard



In [ ]:
# Put your train and validation code here

## Вывод

При какой метрике расстояния и с каким k точность будет максимально точность будет максимальной?


...